# Creación y carga de base de datos en PostgreSQL desde Python

In [ ]:
# Importar paquetes
import pandas as pd 
import psycopg2 

In [ ]:
df = pd.read_csv("../data/halcon_viajes_limpios.csv") # Cargar el archivo

## 1. Crear la base de datos en PostgreSQL

In [ ]:
# Una vez creado el modelo relacional, crear el script SQL para crear las tablas de este en postgreSQL, validando que coincidan los tipos de datos con el dataset 
'''-- Tabla: usuarios
CREATE TABLE usuarios (
    id_usuario VARCHAR PRIMARY KEY,
    edad INT,
    género VARCHAR,
    ubicación VARCHAR,
    tipo_viajero VARCHAR,
    rango_edad VARCHAR
);

-- Tabla: reservas
CREATE TABLE reservas (
    id_reserva VARCHAR PRIMARY KEY,
    id_usuario VARCHAR REFERENCES usuarios(id_usuario),
    fecha_reserva VARCHAR,
    fecha_viaje VARCHAR,
    antelacion_reserva INT,
    duracion_viaje INT,
    numero_personas INT,
    tipo_paquete VARCHAR,
    costo_total NUMERIC,
    numero_noches_estancia INT,
    mes_reserva INT,
    mes_viaje INT,
    estacionalidad VARCHAR,
    estado_reserva VARCHAR,
    cancelacion_reserva VARCHAR,
    promocion_aplicada VARCHAR,
    fuente_reserva VARCHAR
);

-- Tabla: viajes
CREATE TABLE viajes (
    id_reserva VARCHAR PRIMARY KEY REFERENCES reservas(id_reserva),
    destino VARCHAR,
    tipo_alojamiento VARCHAR,
    clase_vuelo VARCHAR,
    actividades_reservadas VARCHAR
);

-- Tabla: pagos
CREATE TABLE pagos (
    id_reserva VARCHAR PRIMARY KEY REFERENCES reservas(id_reserva),
    metodo_pago VARCHAR,
    gasto_por_persona NUMERIC,
    gasto_por_duracion NUMERIC
);

-- Tabla: feedback
CREATE TABLE feedback (
    id_reserva VARCHAR PRIMARY KEY REFERENCES reservas(id_reserva),
    calificacion_usuario INT,
    comentarios VARCHAR
);'''

## 2. Importar los datos desde Python a PostgreSQL

#### Dividir el dataframe original en dataframes por tabla según el modelo relacional creado

In [8]:
# Tabla: usuarios
usuarios_df = df[['id_usuario', 'edad', 'género', 'ubicación', 'tipo_viajero', 'rango_edad']].drop_duplicates()

# Tabla: reservas
reservas_df = df[['id_reserva', 'id_usuario', 'fecha_reserva', 'fecha_viaje', 'antelacion_reserva',
                  'duracion_viaje', 'numero_personas', 'tipo_paquete', 'costo_total',
                  'numero_noches_estancia', 'mes_reserva', 'mes_viaje', 'estacionalidad',
                  'estado_reserva', 'cancelacion_reserva', 'promocion_aplicada', 'fuente_reserva']]

# Tabla: viajes
viajes_df = df[['id_reserva', 'destino', 'tipo_alojamiento', 'clase_vuelo', 'actividades_reservadas']]

# Tabla: pagos
pagos_df = df[['id_reserva', 'metodo_pago', 'gasto_por_persona', 'gasto_por_duracion']]

# Tabla: feedback
feedback_df = df[['id_reserva', 'calificacion_usuario', 'comentarios']]


### Insertar todas las tablas a PostgreSQL

In [ ]:
# 1. Conexión a PostgreSQL
conn = psycopg2.connect(
    dbname="proyecto_viajes_halcon",
    user="postgres",
    password="admin",
    host="localhost",
    port="5432"
)
# 2. Función para insertar cualquier DataFrame
def insertar_dataframe(df, tabla, columnas, conn):
    cursor = conn.cursor()
    registros = list(df.itertuples(index=False, name=None))
    placeholders = ', '.join(['%s'] * len(columnas))
    query = f"""
        INSERT INTO {tabla} ({', '.join(columnas)})
        VALUES ({placeholders})
        ON CONFLICT DO NOTHING;
    """
    cursor.executemany(query, registros)
    conn.commit()
    print(f"Insertados en {tabla}: {cursor.rowcount} filas.")
    cursor.close()
    
# 5. Insertar cada tabla
insertar_dataframe(usuarios_df, "usuarios", [
    "id_usuario", "edad", "género", "ubicación", "tipo_viajero", "rango_edad"
], conn)

insertar_dataframe(reservas_df, "reservas", [
    "id_reserva", "id_usuario", "fecha_reserva", "fecha_viaje", "antelacion_reserva",
    "duracion_viaje", "numero_personas", "tipo_paquete", "costo_total",
    "numero_noches_estancia", "mes_reserva", "mes_viaje", "estacionalidad",
    "estado_reserva", "cancelacion_reserva", "promocion_aplicada", "fuente_reserva"
], conn)

insertar_dataframe(viajes_df, "viajes", [
    "id_reserva", "destino", "tipo_alojamiento", "clase_vuelo", "actividades_reservadas"
], conn)

insertar_dataframe(pagos_df, "pagos", [
    "id_reserva", "metodo_pago", "gasto_por_persona", "gasto_por_duracion"
], conn)

insertar_dataframe(feedback_df, "feedback", [
    "id_reserva", "calificacion_usuario", "comentarios"
], conn)

# 6. Cerrar conexión
conn.close()
print("Inserción completada y conexión cerrada.")

## Documentación: Creación y carga de base de datos en PostgreSQL desde Python
Este documento resume paso a paso el proceso de creación de un modelo relacional para el dataset original, la implementación en PostgreSQL y la inserción automatizada de datos desde Python.
- Se diseñó un modelo relacional con cinco tablas principales (usuarios, reservas, viajes, pagos y feedback).
- Se creó una base de datos en PostgreSQL, y en ella se crearon las tablas manualmente en PostgreSQL con claves primarias y foráneas correspondientes.
- Se preparó el dataset desde Python de acuerdo al modelo relacional. El CSV se dividió en 5 DataFrames temáticos, eliminando duplicados donde correspondía.
- Se utilizó la librería `psycopg2` para establecer la conexión desde Python al motor de base de datos.
- Se creó una función genérica para automatizar la inserción de datos en su tabla correspondiente.
- Se ejecutó la función para cada tabla.
- Resultado: Los datos del dataset fueron correctamente cargados en una base de datos relacional en PostgreSQL, estructurados y listos para ser consultados desde Python y herramientas de BI, optimizando así la gestión de información.

---

# Consultas SQL ejecutadas desde Python orientadas al análisis del negocio

En esta sección se utilizarán **consultas SQL** sobre la base de datos relacional creada en PostgreSQL para:

- Reproducir los **insights clave del análisis exploratorio** realizado previamente en Python
- Demostrar **dominio del lenguaje SQL** para responder preguntas reales del negocio
- Validar que la estructura relacional permite extraer información útil de forma eficiente
- Potenciar el uso de SQL como herramienta fundamental del analista de datos

Las consultas están orientadas a temas como los patrones de cancelación de reservas y satisfacción del cliente.

Cada consulta responderá a una pregunta de negocio específica, y permitirá mostrar cómo se puede utilizar SQL para obtener valor directamente desde la base de datos, sin necesidad de otras herramientas.

In [12]:
# Conexión a PostgreSQL
conn = psycopg2.connect(
    host="localhost",
    dbname="proyecto_viajes_halcon",
    user="postgres",
    password="admin",
    port=5432
)
cursor = conn.cursor()
# Crear una función para ejecutar consultas SQL
def ejecutar_consulta(sql):
    return pd.read_sql_query(sql, conn)

### Consulta 1: ¿Qué porcentaje de reservas en clase económica fueron canceladas en comparación con clases superiores?

In [ ]:
sql = """
SELECT clase_vuelo,
       COUNT(*) AS total_reservas,
       SUM(CASE WHEN cancelacion_reserva = 'si' THEN 1 ELSE 0 END) AS canceladas,
       ROUND(100.0 * SUM(CASE WHEN cancelacion_reserva = 'si' THEN 1 ELSE 0 END) / COUNT(*), 2) AS porcentaje_cancelacion
FROM reservas r
JOIN viajes v ON r.id_reserva = v.id_reserva
GROUP BY clase_vuelo
ORDER BY porcentaje_cancelacion DESC; """
ejecutar_consulta(sql)
# Objetivo: Confirmar si los clientes que reservan en clase económica cancelan con más frecuencia que los de clase ejecutiva o primera.

### Consulta 2: ¿Cómo varía la tasa de cancelación según la antelación de la reserva?

In [ ]:
sql = """
SELECT
    CASE 
        WHEN antelacion_reserva < 15 THEN '0-14 días'
        WHEN antelacion_reserva BETWEEN 15 AND 30 THEN '15-30 días'
        WHEN antelacion_reserva BETWEEN 31 AND 60 THEN '31-60 días'
        ELSE '60+ días'
    END AS rango_antelacion,
    COUNT(*) AS total_reservas,
    SUM(CASE WHEN cancelacion_reserva = 'si' THEN 1 ELSE 0 END) AS canceladas,
    ROUND(100.0 * SUM(CASE WHEN cancelacion_reserva = 'si' THEN 1 ELSE 0 END) / COUNT(*), 2) AS porcentaje_cancelacion
FROM reservas
GROUP BY rango_antelacion
ORDER BY porcentaje_cancelacion DESC;
 """
ejecutar_consulta(sql)
# Objetivo: Comprender si las reservas hechas con poca antelación se cancelan más, lo que puede servir para ajustar las condiciones de cancelación.

### Consulta 3: ¿Cuál es la relación entre la duración del viaje y la tasa de cancelación?

In [ ]:
sql = """
SELECT
    CASE 
        WHEN duracion_viaje <= 3 THEN '1-3 días'
        WHEN duracion_viaje BETWEEN 4 AND 7 THEN '4-7 días'
        WHEN duracion_viaje BETWEEN 8 AND 14 THEN '8-14 días'
        ELSE '15+ días'
    END AS rango_duracion,
    COUNT(*) AS total_reservas,
    SUM(CASE WHEN cancelacion_reserva = 'si' THEN 1 ELSE 0 END) AS canceladas,
    ROUND(100.0 * SUM(CASE WHEN cancelacion_reserva = 'si' THEN 1 ELSE 0 END) / COUNT(*), 2) AS porcentaje_cancelacion
FROM reservas
GROUP BY rango_duracion
ORDER BY porcentaje_cancelacion DESC; """
ejecutar_consulta(sql)
# Objetivo: Permite observar si los viajes más cortos tienen mayor tendencia a cancelarse, lo cual influye en decisiones sobre promociones o penalizaciones.

### Consulta 4: ¿Qué grupo de edad presenta la mayor tasa de cancelación?

In [ ]:
sql = """
SELECT rango_edad,
       COUNT(*) AS total_reservas,
       SUM(CASE WHEN cancelacion_reserva = 'si' THEN 1 ELSE 0 END) AS canceladas,
       ROUND(100.0 * SUM(CASE WHEN cancelacion_reserva = 'si' THEN 1 ELSE 0 END) / COUNT(*), 2) AS porcentaje_cancelacion
FROM reservas r
JOIN usuarios u ON r.id_usuario = u.id_usuario
GROUP BY rango_edad
ORDER BY porcentaje_cancelacion DESC;
 """
ejecutar_consulta(sql)
# Objetivo: Identifica si los jóvenes (19-30 años) cancelan con más frecuencia, lo que ayuda a ajustar la estrategia comercial por segmento etario.

### Consulta 5: ¿Qué tipo de viajero cancela más: turista o negocio?

In [ ]:
sql = """
SELECT tipo_viajero,
       COUNT(*) AS total_reservas,
       SUM(CASE WHEN cancelacion_reserva = 'si' THEN 1 ELSE 0 END) AS canceladas,
       ROUND(100.0 * SUM(CASE WHEN cancelacion_reserva = 'si' THEN 1 ELSE 0 END) / COUNT(*), 2) AS porcentaje_cancelacion
FROM reservas r
JOIN usuarios u ON r.id_usuario = u.id_usuario
GROUP BY tipo_viajero
ORDER BY porcentaje_cancelacion DESC;
 """
ejecutar_consulta(sql)
# Objetivo: Evalúa si los turistas cancelan más que los viajeros de negocios, lo que permite orientar las políticas comerciales y campañas de fidelización.

### Consulta 6: ¿Qué tipo de paquete genera más cancelaciones?

In [ ]:
sql = """
SELECT tipo_paquete,
       COUNT(*) AS total_reservas,
       SUM(CASE WHEN cancelacion_reserva = 'si' THEN 1 ELSE 0 END) AS canceladas,
       ROUND(100.0 * SUM(CASE WHEN cancelacion_reserva = 'si' THEN 1 ELSE 0 END) / COUNT(*), 2) AS porcentaje_cancelacion
FROM reservas
GROUP BY tipo_paquete
ORDER BY porcentaje_cancelacion DESC;
 """
ejecutar_consulta(sql)
# Objetivo: Detecta si el tipo 'solo_vuelo' tiene tasas de cancelación especialmente altas y si debería excluirse de promociones generales.

### Consulta 7: ¿Qué tipo de alojamiento está más asociado a cancelaciones?

In [ ]:
sql = """
SELECT tipo_alojamiento,
       COUNT(*) AS total_reservas,
       SUM(CASE WHEN r.cancelacion_reserva = 'si' THEN 1 ELSE 0 END) AS canceladas,
       ROUND(100.0 * SUM(CASE WHEN r.cancelacion_reserva = 'si' THEN 1 ELSE 0 END) / COUNT(*), 2) AS porcentaje_cancelacion
FROM reservas r
JOIN viajes v ON r.id_reserva = v.id_reserva
GROUP BY tipo_alojamiento
ORDER BY porcentaje_cancelacion DESC;
 """
ejecutar_consulta(sql)
# Objetivo: Permite evaluar si los hoteles tienen una política de cancelación más permisiva que otros alojamientos como Airbnb o resorts.

### Consulta 8: ¿Qué motivos aparecen con más frecuencia en los comentarios de reservas canceladas?

In [ ]:
sql = """
SELECT comentarios,
       COUNT(*) AS cantidad
FROM feedback f
JOIN reservas r ON f.id_reserva = r.id_reserva
WHERE cancelacion_reserva = 'si'
GROUP BY comentarios
ORDER BY cantidad DESC;
 """
ejecutar_consulta(sql)
# Objetivo: Detecta los principales motivos por los cuales los usuarios cancelan, como “problemas con la reserva” o “no me gustó el servicio”.

### Consulta 9: ¿Qué fuente de reserva tiene más cancelaciones?

In [ ]:
sql = """
SELECT fuente_reserva,
       COUNT(*) AS total_reservas,
       SUM(CASE WHEN cancelacion_reserva = 'si' THEN 1 ELSE 0 END) AS canceladas,
       ROUND(100.0 * SUM(CASE WHEN cancelacion_reserva = 'si' THEN 1 ELSE 0 END) / COUNT(*), 2) AS tasa_cancelacion
FROM reservas
GROUP BY fuente_reserva
ORDER BY tasa_cancelacion DESC;
 """
ejecutar_consulta(sql)
# Objetivo: Ayuda a identificar si los usuarios que reservan mediante app móvil o agentes son más propensos a cancelar.

### Consulta 10: ¿Qué tipo de paquete cancelan más los jóvenes (19-30)?

In [ ]:
sql = """
SELECT tipo_paquete,
       COUNT(*) AS total_reservas,
       SUM(CASE WHEN cancelacion_reserva = 'si' THEN 1 ELSE 0 END) AS canceladas,
       ROUND(100.0 * SUM(CASE WHEN cancelacion_reserva = 'si' THEN 1 ELSE 0 END) / COUNT(*), 2) AS porcentaje_cancelacion
FROM reservas r
JOIN usuarios u ON r.id_usuario = u.id_usuario
WHERE rango_edad = '19-30'
GROUP BY tipo_paquete
ORDER BY porcentaje_cancelacion DESC;
"""
ejecutar_consulta(sql)
# Objetivo: Evalúa si los usuarios jóvenes tienden a cancelar más cuando reservan paquetes específicos, como ‘solo_vuelo’.